# Virtual Paint Project (OpenCV)

In [1]:
# work for red and green markers
import cv2
import numpy as np
import time
def empty(a):
    pass
cap=cv2.VideoCapture("http://26.159.20.114:8080/video")# use 0 for default webcam
cap.set(3,1640)#width
cap.set(4,1640)#height
# cap.set(10,1)#brightne
cv2.namedWindow("Trackbars")
cv2.resizeWindow("Trackbars",640,300)
cv2.createTrackbar("Hue Min","Trackbars",0,179,empty)
cv2.createTrackbar("Hue Max","Trackbars",9,179,empty)
cv2.createTrackbar("Sat Min","Trackbars",113,255,empty)
cv2.createTrackbar("Sat Max","Trackbars",251,255,empty)
cv2.createTrackbar("Val Min","Trackbars",157,255,empty)
cv2.createTrackbar("Val Max","Trackbars",255,255,empty)
mycolors=[[0,113,157,9,251,255],[36,55,139,156,255,255]]
mycolorvalues=[[0,0,255],
               [0,255,0],]
mypoints=[]    #[x,y,colorId]


        
def get_contours(img,result_img):
    contours,hierarchy=cv2.findContours(img,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    x,y,w,h=0,0,0,0
    for cnt in contours:
        area=cv2.contourArea(cnt)
        if area>=500:
#             cv2.drawContours(result_img,cnt,-1,(0,0,255),5)
            peri=cv2.arcLength(cnt,True)
            approx=cv2.approxPolyDP(cnt,0.02*peri,True)
            x,y,w,h=cv2.boundingRect(approx)
#             cv2.rectangle(result_img,(x,y),(x+w,y+h),5)
    return x+w//2,y

def find_colors(img,mycolors):
    hsv_img=cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
    count=0
    newPoints=[]
    for color in mycolors:
        lower=np.array(color[:3])
        upper=np.array(color[3:])
        mask=cv2.inRange(hsv_img,lower,upper)
        cv2.imshow("mask",mask)
        x,y=get_contours(mask,img)
        cv2.circle(result_img,(x,y),25,mycolorvalues[count],cv2.FILLED)
        if x!=0 and y!=0:
            newPoints.append([x,y,count])
        count+=1
    return newPoints
#         cv2.imshow(str(color[0]),mask)                  

def drawOnCanvas(mypoints,mycolorvalues):
    for point in mypoints:
        cv2.circle(result_img,(point[0],point[1]),25,mycolorvalues[point[2]],cv2.FILLED)
        
while True:
    success,img=cap.read()
    result_img=img.copy()
    hsv_img=cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
    h_min=cv2.getTrackbarPos("Hue Min","Trackbars")
    h_max=cv2.getTrackbarPos("Hue Max","Trackbars")
    sat_min=cv2.getTrackbarPos("Sat Min","Trackbars")
    sat_max=cv2.getTrackbarPos("Sat Max","Trackbars")
    val_min=cv2.getTrackbarPos("Val Min","Trackbars")
    val_max=cv2.getTrackbarPos("Val Max","Trackbars")
    lower=np.array([h_min,sat_min,val_min])
    upper=np.array([h_max,sat_max,val_max])
    mask=cv2.inRange(hsv_img,lower,upper)
#     cv2.imshow("mask",mask)
    newPoints=find_colors(result_img,mycolors)
    if len(newPoints)!=0:
        for newp in newPoints:
            mypoints.append(newp)
    if len(mypoints)!=0:
        drawOnCanvas(mypoints,mycolorvalues)
    cv2.imshow("result_image",result_img)
    if cv2.waitKey(1) & 0xFF ==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 